## Installing Necessary Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.

# Dataset details
Instacart Data can be downloaded from [here](https://www.kaggle.com/competitions/instacart-market-basket-analysis/data). We just need product & department csv files

In [3]:
import pandas as pd
df_product = pd.read_csv("/content/drive/MyDrive/products.csv")
df_dept = pd.read_csv('/content/drive/MyDrive/departments.csv')

In [4]:
df_joined = pd.merge(df_product, df_dept, on = ['department_id'])
df_joined['text'] = df_joined.apply(lambda row: row['product_name'] + " ->: " + row['department'], axis = 1)

In [5]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_joined, test_size=0.2, random_state=42)

In [6]:
train_df.head(10)

,product_id,product_name,aisle_id,department_id,department,text
7361,10376,Organic Caraway Seeds,104,13,pantry,Organic Caraway Seeds ->: pantry
32534,40131,Original Laundry Detergent,75,17,household,Original Laundry Detergent ->: household
1457,11913,Shelled Pistachios,117,19,snacks,Shelled Pistachios ->: snacks
5201,41392,Harvest Berry Chewy Granola Bars,3,19,snacks,Harvest Berry Chewy Granola Bars ->: snacks
38539,28627,Veganic Sprouted Red Fife Raisin Bran,121,14,breakfast,Veganic Sprouted Red Fife Raisin Bran ->: brea...
18377,29407,Root Vegetable Cakes,42,1,frozen,Root Vegetable Cakes ->: frozen
28627,29571,Alta Dena 1% Milk,84,16,dairy eggs,Alta Dena 1% Milk ->: dairy eggs
30283,3589,Laundry Detergent Pods,75,17,household,Laundry Detergent Pods ->: household
32030,31685,"Daily Shower Cleaner Scrub Free Refill, Fresh ...",114,17,household,"Daily Shower Cleaner Scrub Free Refill, Fresh ..."
28655,30119,Organic French Style Meyer Lemon Yogurt,120,16,dairy eggs,Organic French Style Meyer Lemon Yogurt ->: da...


In [7]:
test_df.head(10)

,product_id,product_name,aisle_id,department_id,department,text
33626,24154,Free & Clear Stage 4 Overnight Diapers,56,18,babies,Free & Clear Stage 4 Overnight Diapers ->: babies
18192,27348,"Beef pot roast with roasted potatoes, carrots,...",38,1,frozen,"Beef pot roast with roasted potatoes, carrots,..."
47099,27181,Coffee Liquer,124,5,alcohol,Coffee Liquer ->: alcohol
48183,20577,Bread Rolls,43,3,bakery,Bread Rolls ->: bakery
22197,16472,French Milled Oval Almond Gourmande Soap,25,11,personal care,French Milled Oval Almond Gourmande Soap ->: p...
31573,24121,Dust Pan,114,17,household,Dust Pan ->: household
45362,5477,Roasted Pine Nut Hommus,67,20,deli,Roasted Pine Nut Hommus ->: deli
14131,27921,Cranberry Raspberry Juice Cocktail,98,7,beverages,Cranberry Raspberry Juice Cocktail ->: beverages
26903,4786,Sweet Cream Butter Salted,36,16,dairy eggs,Sweet Cream Butter Salted ->: dairy eggs
39417,8796,Traditional Chicken Barley Soup,69,15,canned goods,Traditional Chicken Barley Soup ->: canned goods


In [8]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df),
})

## Loading the model

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Let's also load the tokenizer below

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

**Let's check what the base model predicts before finetuning. :)**

In [11]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


sequences = pipeline(
   ["“Free & Clear Stage 4 Overnight Diapers” ->:","Bread Rolls ->:","French Milled Oval Almond Gourmande Soap ->:"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Result: “Free & Clear Stage 4 Overnight Diapers” ->: 2019-06-10 13:49:57
 фев 15, 2018... 520. 521. 522. 523. 524. 525. 526. 527. 528. 529.... 3210. 3211. 3212. 3213. 3214. 3215. 3216. 3217. 3218. 3219. 3220. 3221..... 520. 521. 522. 523. 524. 525
Result: Bread Rolls ->: 2. Unterscheidung zwischen Back- und Brotlaube.
* 1796: Johann Georg Krämer: Kleines Lehrbuch der Bäckerkunst. ->: Der Brotlauf.
* 1804: Johann Georg Krämer: Vollständiges Lehr- und Handbuch der Bäckerkunst. ->: Der Brotlauf.
* 1851: Julius Kollath: Handwerks-Lexikon. Brotbäcker, Brotweber.
* 1929: Wilhelm Fuchs: Die Bäcker- und Bäckereiwirtschaft im Deutschen Reich. Brotlaube.
* 1937: Wilhelm Fuchs: Handbuch für Bäcker. Brotlaube.
* 1972: Heinz Döhnert: Die Brotlaube. Einführung in die
Result: French Milled Oval Almond Gourmande Soap ->: https://www. nobody is perfect.
It is the best and fastest way to get rid of those dark circles under our eyes. This is a good one. I am a huge fan of this soap.
The first soap was ma

Below we will load the configuration file in order to create the LoRA model. According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance.

In [12]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [13]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [14]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/39750 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [15]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [16]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.029100
2,3.089600
3,4.359200
4,6.271200
5,4.250100
6,3.057500
7,4.467000
8,3.854300
9,4.339800
10,5.121500


TrainOutput(global_step=120, training_loss=2.489347438017527, metrics={'train_runtime': 686.9412, 'train_samples_per_second': 2.795, 'train_steps_per_second': 0.175, 'total_flos': 627303342243840.0, 'train_loss': 2.489347438017527, 'epoch': 0.05})

In [17]:
lst_test_data = list(test_df['text'])

In [18]:
len(lst_test_data)

9938

In [19]:
sample_size = 25
lst_test_data_short = lst_test_data[:sample_size]

In [20]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    lst_test_data_short,
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


0 Free & Clear Stage 4 Overnight Diapers ->: babies ->: diapers & wipes ->: disposable diapers ->: diapers ->: overnight diapers ->: overnight diapers ->: personal care products ->: babies personal care products personal care products ->: diapers & wipes personal care products personal care products personal care products personal care products ->: overnight diapers personal care products personal care products personal care products personal care products ->:
1 Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: frozen foods meat seafood & poultry ->: meats beef ->: beef products ->: frozen foods meat seafood & poultry meat seafood & poultry beef ->: beef products pork & beef products beef products beef pot ro
2 Coffee Liquer ->: alcohol beverages liquor ->: adult beverages ->: liquor ->: coffee liqueur original ->: non-alcoholic beverages personal care household goods personal care household goods personal care household goods

In [21]:
def correct_answer(ans):
  return (ans.split("->:")[1]).strip()

answers = []
for ix,seq in enumerate(sequences):
    # print(ix,seq[0]['generated_text'])
    answers.append(correct_answer(seq[0]['generated_text']))

answers

['babies',
 'frozen foods meat seafood & poultry',
 'alcohol beverages liquor',
 'bakery & dairy eggs & dairy eggs & dairy eggs & dairy eggs & dairy eggs',
 'personal care personal wash personal hygiene',
 'household essentials kitchen utensils baking & kitchen tools baking tools & gadgets dust pans & brooms',
 'deli cheese eggs',
 'beverages other drinks juices cranberry raspberry juice cocktail',
 'dairy eggs dairy eggs eggs dairy eggs dairy eggs butter dairy eggs eggs dairy eggs dairy eggs',
 'canned goods canned goods: soup',
 'dairy eggs dairy eggs other dairy beverages almond dairy milk almond dairy milk almond cashew',
 'snacks bars & nuts',
 'dry goods pasta rice pasta sauces pasta side dishes international',
 'personal care personal care babies & kids personal care baby care babies',
 'beverages alcoholic beverages',
 'international aisles international products & gourmet foods',
 'breakfast cereals pasta, rice, beans & grains other pasta, rice, beans & grains pasta, rice, bea

In [22]:
df_evaluate = test_df.iloc[:sample_size][['product_name','department']]

df_evaluate = df_evaluate.reset_index(drop=True)

df_evaluate['department_predicted'] = answers

df_evaluate

,product_name,department,department_predicted
0,Free & Clear Stage 4 Overnight Diapers,babies,babies
1,"Beef pot roast with roasted potatoes, carrots,...",frozen,frozen foods meat seafood & poultry
2,Coffee Liquer,alcohol,alcohol beverages liquor
3,Bread Rolls,bakery,bakery & dairy eggs & dairy eggs & dairy eggs ...
4,French Milled Oval Almond Gourmande Soap,personal care,personal care personal wash personal hygiene
5,Dust Pan,household,household essentials kitchen utensils baking &...
6,Roasted Pine Nut Hommus,deli,deli cheese eggs
7,Cranberry Raspberry Juice Cocktail,beverages,beverages other drinks juices cranberry raspbe...
8,Sweet Cream Butter Salted,dairy eggs,dairy eggs dairy eggs eggs dairy eggs dairy eg...
9,Traditional Chicken Barley Soup,canned goods,canned goods canned goods: soup
